In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [2]:
txtFile=sc.textFile('../data/covtype.csv')
#Convert it into RDD of lists 
rdd=(txtFile.map(lambda x:x.split())
    .map(lambda x: x[0].strip("'").split(","))
    .map(lambda x:[float(v) for v in x])
    .map(lambda x: (x[-1]-1,x[0:-1])))

In [11]:
import itertools
soil_list =[]
for k in range(40):
    string = 'Soil_Type_' + str(k+1)
    soil_list.append(string)
WA_list =[]
for k in range(4):
    string = 'WA_' + str(k+1)
    WA_list.append(string)
names = [['Elevation'], ['Aspect'], ['Slope'], ['HDHyrdo'], ['VDHydro'], ['HDRoadways'], \
         ['9amHills'],['NoonHills'], ['3pmHills'], ['HDFirePoints'], WA_list,\
         soil_list, ['Cover_Type']]
columns = list(itertools.chain(*names))

In [13]:
import random

In [29]:
#selecting random features
m = 10 #No of features
indices = range(len(columns)-1)
random.shuffle(indices)
sampled_c = [columns[j] for j in indices[:m]]

In [36]:
#Code for implementing in rdd
%time sampled_rdd = rdd.map(lambda x: (x[0],[x[1][j] for j in indices[:m]]))

CPU times: user 33 µs, sys: 8 µs, total: 41 µs
Wall time: 41 µs


In [57]:
#Imports - move later to top
import numpy as np

In [58]:
#total counts of the data
t_count = rdd.count()

In [87]:
##Implementing the ID3 and testing the Entropy
test_index = 0
%time entrop_score = entropy_score(sampled_rdd,test_index)
%time entrop_score = entrop_score.take(1)[0][1]

CPU times: user 19.9 ms, sys: 5.2 ms, total: 25.1 ms
Wall time: 40 ms
CPU times: user 8.84 ms, sys: 2.15 ms, total: 11 ms
Wall time: 8 s


In [96]:
def entropy_score(sampled_rdd,index):
    new_rdd = sampled_rdd.map(lambda x:x[1][index]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[0],(-x[1]/t_count)*(np.log2(x[1]/t_count))))\
            .map(lambda x:(index,x[1])).reduceByKey(lambda a,b:a+b)
    return new_rdd

In [101]:
def get_all_scores(sampled_rdd,m):
    new_rdd = entropy_score(sampled_rdd,0)
    for idx in range(1,m):
        rdd_to_join = entropy_score(sampled_rdd,idx)
        new_rdd = new_rdd.union(rdd_to_join)
    return new_rdd

In [92]:
%time new_rdd = sampled_rdd.map(lambda x:x[1][test_index]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[0],(-x[1]/t_count)*(np.log2(x[1]/t_count))))\
            .map(lambda x:(1,x[1])).reduceByKey(lambda a,b:a+b)

CPU times: user 17.8 ms, sys: 7.69 ms, total: 25.5 ms
Wall time: 46.3 ms


In [102]:
%time print get_all_scores(sampled_rdd,m).collect()

[(0, 0.71862713465838812), (3, 0.22658524039946459), (6, 0.089399302504473782), (9, 7.2756316788852136), (1, 0.11272233723151603), (4, 4.8521449798995224), (7, 0.019634971549216975), (2, 0.069306853444915606), (5, 0.19439738190120537), (8, 0.044623723975760712)]
CPU times: user 157 ms, sys: 35.2 ms, total: 192 ms
Wall time: 1min 25s


In [99]:
rdd_to_join = entropy_score(sampled_rdd,1)